In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2 as psc

In [2]:
#Creamos las credenciales para ingresar a la base de datos
from sqlalchemy import create_engine
server = {
    'user' : 'postgres',
    'password' : 'detectivecreador',
    'host' : 'localhost',
    'port' : '5434',
    'database' : 'SITEC, soluciones y servicios'
}

#Creamos la cadena de conexión
#conn_str = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

conn_str = f"postgresql+psycopg2://{server['user']}:{server['password']}@{server['host']}:{server['port']}/{server['database']}"

#Creamos el motor de conexión

engine = create_engine(conn_str)

In [3]:
#Extraemos la tabla compras
#Tenemos que castear la columna importe total ya que esta en formato money y al extraerlo y guardarlo en un dataframe, el tipo
#de columna será tipo str
consulta = '''SELECT id_compra, 
            id_proveedor, 
            id_estado, status, 
            ref_proveedor, 
            fecha_recepcion AS fecha, 
            fecha_cancelacion,
            CAST(importe_total AS numeric) AS importe_total FROM compras'''

compras = pd.read_sql(consulta,engine)


In [4]:
#Revisamos las variables estadisticas del dataframe
compras.describe()

 

,id_compra,id_proveedor,id_estado,importe_total
count,9910.000000,9910.000000,9910.000000,9.910000e+03
mean,4955.500000,688.875177,1.617154,1.848554e+04
std,2860.914918,392.101356,0.713287,8.533682e+04
min,1.000000,1.000000,1.000000,7.840000e+00
25%,2478.250000,422.000000,1.000000,5.370275e+02
50%,4955.500000,727.000000,1.000000,1.776625e+03
75%,7432.750000,935.000000,2.000000,1.102000e+04
max,9910.000000,14001.000000,3.000000,5.271941e+06


Si queremos realizar queries dentro de un dataframe, tenemos que importar la libreria *pandassql*:

**pip install pandasql**



In [6]:
import pandasql as ps

#Definimos la query

consulta = '''SELECT importe_total FROM compras'''

print(ps.sqldf(consulta,locals())) #la función .sqldf nos permite realizar una consulta (primer argumento) a los dataframes locales
#la tabla que va a tomar es la que viene en la querie, detectará ese nombre y buscara en el area local un dataframe con ese nombre

      importe_total
0            449.00
1            393.18
2            180.00
3           4505.93
4            123.00
...             ...
9905       12819.70
9906         301.00
9907       38860.00
9908        7984.59
9909       29997.38

[9910 rows x 1 columns]


In [7]:
#Extraemos las tablas que existen en la base de datos.
qproveedores = 'SELECT * FROM proveedores'
proveedores = pd.read_sql(qproveedores,engine)


In [121]:
##Obtener los 3 mejores proveedores por mes del 2024
enero = '''SELECT DISTINCT(p.nombre), SUM(c.importe_total) AS importe__proveedor FROM compras c
        INNER JOIN proveedores p
        ON c.id_proveedor = p.id_proveedor
        WHERE fecha BETWEEN '2024-01-01T00:00:00.000Z' AND '2024-01-31T00:00:00.000Z'
        GROUP BY p.nombre
        ORDER BY SUM(c.importe_total) DESC
        LIMIT 3
        '''
febrero = '''SELECT DISTINCT(p.nombre), SUM(c.importe_total) AS importe__proveedor FROM compras c
        INNER JOIN proveedores p
        ON c.id_proveedor = p.id_proveedor
        WHERE fecha BETWEEN '2024-02-01T00:00:00.000Z' AND '2024-02-31T00:00:00.000Z'
        GROUP BY p.nombre
        ORDER BY SUM(c.importe_total) DESC
        LIMIT 3
        '''
marzo = '''SELECT DISTINCT(p.nombre), SUM(c.importe_total) AS importe__proveedor FROM compras c
        INNER JOIN proveedores p
        ON c.id_proveedor = p.id_proveedor
        WHERE fecha BETWEEN '2024-03-01T00:00:00.000Z' AND '2024-03-31T00:00:00.000Z'
        GROUP BY p.nombre
        ORDER BY SUM(c.importe_total) DESC
        LIMIT 3
        '''
abril = '''SELECT DISTINCT(p.nombre), SUM(c.importe_total) AS importe__proveedor FROM compras c
        INNER JOIN proveedores p
        ON c.id_proveedor = p.id_proveedor
        WHERE fecha BETWEEN '2024-04-01T00:00:00.000Z' AND '2024-04-31T00:00:00.000Z'
        GROUP BY p.nombre
        ORDER BY SUM(c.importe_total) DESC
        LIMIT 3
        '''
mayo = '''SELECT DISTINCT(p.nombre), SUM(c.importe_total) AS importe__proveedor FROM compras c
        INNER JOIN proveedores p
        ON c.id_proveedor = p.id_proveedor
        WHERE fecha BETWEEN '2024-05-01T00:00:00.000Z' AND '2024-05-31T00:00:00.000Z'
        GROUP BY p.nombre
        ORDER BY SUM(c.importe_total) DESC
        LIMIT 3
        '''

meses = ['enero','febrero','marzo','abril','mayo']
q = [ps.sqldf(enero,locals()),ps.sqldf(febrero,locals()),ps.sqldf(marzo,locals()),ps.sqldf(abril,locals()),ps.sqldf(mayo,locals())]


In [9]:
rankingproveedores2024 = {}
rankingproveedores2024 = {mes : consulta for mes,consulta in zip(meses,q)}

In [11]:
rankingproveedores2024['febrero']

,nombre,importe__proveedor
0,DISTRIBUIDORA DE MAT. ELECTRICOS IND. DE TOLUC...,590173.58
1,"Zigor Soluciones Integrales, S.A. de C.V.",509356.06
2,"Productos Eléctricos y Ferreteros, S.A. de C.V...",243632.83


In [29]:
# %, cantidad de compras por almacen mensuales


qestados = '''SELECT * FROM estados'''
estados = pd.read_sql(qestados, engine)

q = '''SELECT  e.estado, 
               SUM(c.importe_total)/(SELECT SUM(importe_total) FROM compras WHERE fecha BETWEEN '2024-05-01T00:00:00.000Z' AND '2024-05-31T00:00:00.000Z') AS porcentaje, 
               SUM(c.importe_total) AS importe_por_estado FROM compras c
        INNER JOIN estados e
        ON c.id_estado = e.id_estado
        WHERE fecha BETWEEN '2024-05-01T00:00:00.000Z' AND '2024-05-31T00:00:00.000Z'           --Mayo
        GROUP BY e.estado
        ORDER BY importe_por_estado DESC'''


print(ps.sqldf(q,locals()))

            estado  porcentaje  importe_por_estado
0        Queretaro    0.978747          4232307.98
1  San Luis Potosi    0.019646            84951.76
2       Guanajuato    0.001608             6952.00


In [50]:
# corroborar las relación entre cancelación y status, tienen que empatar

q_can_stat = '''SELECT status, fecha_cancelacion FROM compras
                WHERE status = 'Cancelada' AND fecha_cancelacion IS NULL '''
                
print(ps.sqldf(q_can_stat,locals()))               

#Si existe correlación 

Empty DataFrame
Columns: [status, fecha_cancelacion]
Index: []


In [109]:
#Porcentaje de canceladas
q_can = """SELECT status,COUNT(status) AS cantidad, CAST(COUNT(status) AS float)*100/(SELECT COUNT(*) FROM compras) AS percent FROM compras
            GROUP BY status"""          #Es necesario castear la operacion aritmetica ya sea multiplicando por un flotannte
                                        #utilizando :: o CAST()
print(ps.sqldf(q_can,locals()))    

#Otro ejemplo:       
# SELECT status, 
#                   COUNT(status) AS cantidad, 
#                   (COUNT(status) * 100.0 / total.total_count) AS porcentaje_por_concepto
# FROM 
#     compras,(SELECT COUNT(*) AS total_count FROM compras) AS total
# GROUP BY 
#     status, total.total_count


      status  cantidad    percent
0  Cancelada       474   4.783047
1    Emitida       238   2.401615
2   Original      9198  92.815338


# Gráficas

utilizaremos tanto **matplotlib** como **seaborn** dependiendo de la necesidad y la estética

In [171]:
#Graficamos los 3 mejores proveedores en un periodo de 6 meses y un periodo anual
#Extraemos una tabla unicamente con la información necesaria y agregamos una columna que diga el mes para filtrar 


#print(compras[['id_proveedor','fecha','importe_total']])




q_prov = '''SELECT p.nombre,fecha_recepcion AS fecha, MONTH(fecha),  c.importe_total FROM compras c
            INNER JOIN proveedores p
            ON c.id_proveedor = p.id_proveedor'''
            
qsub = '''SELECT id_proveedor,importe_total, EXTRACT(MONTH FROM fecha_recepcion) AS mes FROM compras
          --WHERE MONTH(CAST(fecha_recepcion AS date)) = 1 '''            

print(pd.read_sql(qsub,engine))


#fig, ax = plt.subplots()
#ax.scatter(x[])      

      id_proveedor importe_total  mes
0              660       $449.00  4.0
1              128       $393.18  2.0
2              197       $180.00  2.0
3              243     $4,505.93  2.0
4              174       $123.00  2.0
...            ...           ...  ...
9905           146    $12,819.70  6.0
9906           511       $301.00  6.0
9907          1067    $38,860.00  6.0
9908           151     $7,984.59  6.0
9909          1601    $29,997.38  6.0

[9910 rows x 3 columns]
